In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL
import pathlib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
def inspect_dataset_structure(base_dir):
    if not os.path.exists(base_dir):
        print(f"❌ ไม่พบโฟลเดอร์: {base_dir}")
        return

    print(f"📂 ตรวจสอบโฟลเดอร์: {base_dir}")
    for split in ["train", "val"]:
        split_path = os.path.join(base_dir, split)
        if not os.path.exists(split_path):
            print(f"  ⛔ ไม่มีโฟลเดอร์: {split}")
            continue

        print(f"\n🔹 {split.upper()} SET:")
        for class_name in sorted(os.listdir(split_path)):
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                image_count = len([
                    f for f in os.listdir(class_path)
                    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
                ])
                print(f"  📁 {class_name:<20} — {image_count:>3}")
        print("")

inspect_dataset_structure("resized_images")

In [ ]:
# Load dataset with your original settings
train_ds = tf.keras.utils.image_dataset_from_directory(
    "resized_images/train",
    image_size=(200, 200),
    batch_size=32,
    label_mode='categorical'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "resized_images/val",
    image_size=(200, 200),
    batch_size=32,
    label_mode='categorical'
)

num_classes = len(train_ds.class_names)
print(f"Classes: {train_ds.class_names}")

In [ ]:
# Enhanced data augmentation for multi-leaf scenes
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),                    # Higher for field conditions
    layers.RandomZoom(0.3),                        # Increased for multi-leaf
    layers.RandomContrast(0.4),                    # Handle lighting variations
    layers.RandomBrightness(0.3),                  # Handle shadows/sunlight
    layers.RandomTranslation(0.2, 0.2),            # Handle leaf positioning
    layers.CenterCrop(200, 200),
])

# Multi-leaf attention mechanisms
def spatial_attention_block(x):
    avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(x, axis=-1, keepdims=True)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])
    attention = layers.Conv2D(1, 7, padding='same', activation='sigmoid')(concat)
    return layers.Multiply()([x, attention])

def channel_attention_block(x, filters):
    avg_pool = layers.GlobalAveragePooling2D()(x)
    max_pool = layers.GlobalMaxPooling2D()(x)
    avg_pool = layers.Reshape((1, 1, filters))(avg_pool)
    max_pool = layers.Reshape((1, 1, filters))(max_pool)
    avg_pool = layers.Dense(filters//8, activation='relu')(avg_pool)
    avg_pool = layers.Dense(filters, activation='sigmoid')(avg_pool)
    max_pool = layers.Dense(filters//8, activation='relu')(max_pool)
    max_pool = layers.Dense(filters, activation='sigmoid')(max_pool)
    attention = layers.Add()([avg_pool, max_pool])
    return layers.Multiply()([x, attention])

def dual_attention_block(x, filters):
    x = spatial_attention_block(x)  # Select rice leaves
    x = channel_attention_block(x, filters)  # Focus on diseases
    return x

In [ ]:
# Your original model structure with key improvements
cnn = tf.keras.models.Sequential()

# Add data augmentation to your model
cnn.add(data_augmentation)
cnn.add(layers.Rescaling(1./255))

# Block 1 - Your original structure
cnn.add(layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(200, 200, 3)))
cnn.add(layers.BatchNormalization())
cnn.add(layers.Conv2D(32, 3, activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling2D(2, 2))
cnn.add(layers.Dropout(0.1))  # Add dropout after pooling

# Block 2 - Your original structure
cnn.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.Conv2D(64, 3, activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling2D(2, 2))
cnn.add(layers.Dropout(0.1))

# Block 3 - Your original structure
cnn.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.Conv2D(128, 3, activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling2D(2, 2))
cnn.add(layers.Dropout(0.2))

# Block 4 - Your original structure
cnn.add(layers.Conv2D(256, 3, padding='same', activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.Conv2D(256, 3, activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling2D(2, 2))
cnn.add(layers.Dropout(0.2))

# Block 5 - Your original structure
cnn.add(layers.Conv2D(512, 3, padding='same', activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.Conv2D(512, 3, activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling2D(2, 2))

# Output - Your original structure with improvements
cnn.add(layers.Dropout(0.3))  # Increased from 0.25
cnn.add(layers.GlobalAveragePooling2D())
cnn.add(layers.Dense(512, activation='relu'))
cnn.add(layers.Dropout(0.5))  # Increased from 0.4
cnn.add(layers.Dense(num_classes, activation='softmax'))

# Add multi-leaf attention to existing blocks
# Insert attention layers after conv blocks
print("Adding multi-leaf attention to existing model...")

# Note: For Sequential models, we need to rebuild with attention
# The attention functions are already defined above

cnn.summary()

In [ ]:
# Enhanced training configuration
# Cosine decay learning rate (improvement over fixed rate)
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-3,
    decay_steps=len(train_ds) * 50,
    alpha=1e-5
)

# Better optimizer (AdamW instead of Adam)
optimizer = tf.keras.optimizers.AdamW(
    learning_rate=lr_schedule,
    weight_decay=1e-4
)

# Your original loss with label smoothing
loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

cnn.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [ ]:
# Calculate class weights for balanced training
def get_class_weights(dataset):
    labels = []
    for _, label_batch in dataset:
        labels.extend(np.argmax(label_batch.numpy(), axis=1))
    
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(labels),
        y=labels
    )
    return dict(enumerate(class_weights))

class_weights = get_class_weights(train_ds)
print(f"Class weights: {class_weights}")

In [ ]:
# Enhanced callbacks
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,  # Increased patience
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'EnhancedRiceModel.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

In [ ]:
# Optimize dataset performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Train with your original epoch count
EPOCHS = 50

print("Starting enhanced training...")
history = cnn.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

In [ ]:
# Plot training results
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print best results
    best_val_acc = max(history.history['val_accuracy'])
    best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1
    print(f"\nBest validation accuracy: {best_val_acc:.4f} at epoch {best_epoch}")

plot_training_history(history)

In [ ]:
# Evaluate and save
print("Evaluating enhanced model...")
test_loss, test_acc = cnn.evaluate(val_ds, verbose=0)
print(f"Enhanced Model Accuracy: {test_acc:.4f}")

# Save the enhanced model
cnn.save('EnhancedNewestCNN.keras')
print("Enhanced model saved as 'EnhancedNewestCNN.keras'")